<a href="https://colab.research.google.com/github/lsteffenel/pyspark-binder/blob/master/02-spark-dataframes-fr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spark SQL: DataFrames
Dans ce notebook, nous allons nous intéresser à l'API SQL Spark. Nous verrons comment utiliser les DataFrames et SQL pour effectuer des opérations courantes de traitement de données.

Pour commencer, nous allons créer un `SparkSession`, qui est une demande d'accès au framework Spark (un peu comme ouvrir un ticket pour un service de dépannage informatique).

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

## DataFrames à partir de collections Python

Dans le cours nous avons fait un petit exemple où nous avons crée un DataFrame à partir d'un fichier **csv**. Il est aussi possible de créer un DataFrame à partir d'une liste Python existante. En plus de la liste elle-même, nous allons également décrire (en partie) la structure des données en nommant les colonnes. En outre, nous pourrions spécifier les types de données des colonnes, mais dans ce cas, nous pouvons laisser Spark déduire cela automatiquement.

Tout d'abord, une liste de tuples en Python est créée, appelée `phone_stock`. Ensuite, nous créons une liste appelée `columns` qui contient le nom de toutes les colonnes du DataFrame. Puis nous utilisons ces deux listes comme entrée pour [`createDataFrame`](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.SparkSession.createDataFrame.html). Le résultat est le DataFrame `phone_df`. Ensuite, nous imprimons le type de `phone_stock` et de `phone_df`.


In [2]:
phone_stock = [
    ('iPhone 6', 'Apple', 6, 549.00),
    ('iPhone 6s', 'Apple', 5, 585.00),
    ('iPhone 7', 'Apple', 11, 739.00),
    ('Pixel', 'Google', 8, 859.00),
    ('Pixel XL', 'Google', 2, 959.00),
    ('Galaxy S7', 'Samsung', 10, 539.00),
    ('Galaxy S6', 'Samsung', 5, 414.00),
    ('Galaxy A5', 'Samsung', 7, 297.00),
    ('Galaxy Note 7', 'Samsung', 0, 841.00)
]

columns = ['model', 'brand', 'stock', 'unit_price']

phone_df = spark.createDataFrame(phone_stock, columns)

print('the type of phoneStock: ' + str(type(phone_stock)))
print('the type of phone_df: ' + str(type(phone_df)))

the type of phoneStock: <class 'list'>
the type of phone_df: <class 'pyspark.sql.dataframe.DataFrame'>


Pour afficher quelques lignes d'un DataFrame, utilisez [`show()`](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.show.html). Par défaut, il affiche 20 lignes, mais vous pouvez donner le nombre de lignes que vous souhaitez voir comme argument.

In [3]:
phone_df.show()

+-------------+-------+-----+----------+
|        model|  brand|stock|unit_price|
+-------------+-------+-----+----------+
|     iPhone 6|  Apple|    6|     549.0|
|    iPhone 6s|  Apple|    5|     585.0|
|     iPhone 7|  Apple|   11|     739.0|
|        Pixel| Google|    8|     859.0|
|     Pixel XL| Google|    2|     959.0|
|    Galaxy S7|Samsung|   10|     539.0|
|    Galaxy S6|Samsung|    5|     414.0|
|    Galaxy A5|Samsung|    7|     297.0|
|Galaxy Note 7|Samsung|    0|     841.0|
+-------------+-------+-----+----------+



Dans les DataFrame Spark, l'exécution n'est pas immédiate (on appelle ça une exécution *lazy* ou paresseuse) : on attend l'enchaînement des opérations pour essayer d'optimiser l'exécution. Le traitement s'effectue seulement quand faisons une action comme [`collect()`](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.collect.html).

Dans le cas ci-dessous, nous obtenons des objets `Row` qui contiennent des paires de noms de colonnes et de valeurs. En effet, le résultat d'une action `collect()` est une structure de données Python (une liste d'objets `Row` dans cet exemple).

In [4]:
all_phones = phone_df.collect()
all_phones

[Row(model='iPhone 6', brand='Apple', stock=6, unit_price=549.0),
 Row(model='iPhone 6s', brand='Apple', stock=5, unit_price=585.0),
 Row(model='iPhone 7', brand='Apple', stock=11, unit_price=739.0),
 Row(model='Pixel', brand='Google', stock=8, unit_price=859.0),
 Row(model='Pixel XL', brand='Google', stock=2, unit_price=959.0),
 Row(model='Galaxy S7', brand='Samsung', stock=10, unit_price=539.0),
 Row(model='Galaxy S6', brand='Samsung', stock=5, unit_price=414.0),
 Row(model='Galaxy A5', brand='Samsung', stock=7, unit_price=297.0),
 Row(model='Galaxy Note 7', brand='Samsung', stock=0, unit_price=841.0)]

Il y a plusieurs façons d'examiner la structure d'un DataFrame : `printSchema`, `schema` et `describe`. [`printSchema`](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.printSchema.html) est particulièrement utile avec les structures imbriquées compliquées, parce qu'il fournit une forme lisible :

In [5]:
phone_df.printSchema()

root
 |-- model: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- stock: long (nullable = true)
 |-- unit_price: double (nullable = true)



Notez que toutes les colonnes sont répertoriées, avec leur type et une valeur booléenne qui indique si la valeur de cette colonne peut être NULL.

[`describe`](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.describe.html) calcule des statistiques sommaires pour les colonnes numériques et les chaînes de caractères :

In [6]:
phone_df.describe().show()

+-------+---------+-------+------------------+------------------+
|summary|    model|  brand|             stock|        unit_price|
+-------+---------+-------+------------------+------------------+
|  count|        9|      9|                 9|                 9|
|   mean|     NULL|   NULL|               6.0| 642.4444444444445|
| stddev|     NULL|   NULL|3.5355339059327378|220.82295573100586|
|    min|Galaxy A5|  Apple|                 0|             297.0|
|    max| iPhone 7|Samsung|                11|             959.0|
+-------+---------+-------+------------------+------------------+



## Extraction de données

Maintenant que nous avons nos données dans un DataFrame, nous voulons l'utiliser pour manipuler les données. Commençons par sélectionner des sous-ensembles de données : des colonnes et/ou des lignes spécifiques.

### Sélection de colonnes

Souvent, toutes les colonnes de nos données ne nous intéressent pas. Les DataFrames permettent de sélectionner très facilement un sous-ensemble en utilisant la méthode [`select`](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.select.html). Il faut savoir que nous ne modifions pas le DataFrame original, mais que nous en créons un nouveau.

In [7]:
# Select only the model column
model_df = phone_df.select("model")
model_df.show()

+-------------+
|        model|
+-------------+
|     iPhone 6|
|    iPhone 6s|
|     iPhone 7|
|        Pixel|
|     Pixel XL|
|    Galaxy S7|
|    Galaxy S6|
|    Galaxy A5|
|Galaxy Note 7|
+-------------+



In [8]:
# Select both the brand and model columns
bm_df = phone_df.select('brand', 'model')
bm_df.show()

+-------+-------------+
|  brand|        model|
+-------+-------------+
|  Apple|     iPhone 6|
|  Apple|    iPhone 6s|
|  Apple|     iPhone 7|
| Google|        Pixel|
| Google|     Pixel XL|
|Samsung|    Galaxy S7|
|Samsung|    Galaxy S6|
|Samsung|    Galaxy A5|
|Samsung|Galaxy Note 7|
+-------+-------------+



## Exercice 1
Sélectionner les colonnes `model` et `stock` de `phone_df`:

In [ ]:
# TODO: Remplacer <FILL IN> avec le code approprié
ms_df = phone_df.<FILL_IN>
ms_df.show()

### Filtrage des lignes

Nous pouvons filtrer des lignes spécifiques en utilisant la méthode DataFrame [`filter`](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.filter.html). Veuillez noter que la méthode [`where`](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.where.html) est un alias de `filter`. Les spécifications des colonnes sont les mêmes que pour la méthode select :

In [9]:
# Sélectionner des lignes avec des téléphones Google
google_df = phone_df.filter(phone_df['brand'] == 'Google')

google_df.show()

+--------+------+-----+----------+
|   model| brand|stock|unit_price|
+--------+------+-----+----------+
|   Pixel|Google|    8|     859.0|
|Pixel XL|Google|    2|     959.0|
+--------+------+-----+----------+



## Exercice 2
Sélectionner les lignes avec `unit_price` inférieur à 550.00

In [ ]:
# TODO: Remplacer <FILL IN> avec le code approprié

cheap_df = phone_df.filter(<FILL IN>)
cheap_df.show()

Des conditions de filtrage multiples peuvent être spécifiées à l'aide des [opérations booléennes](https://docs.python.org/3/library/stdtypes.html#boolean-operations-and-or-not) de Python :

In [10]:
phone_df.filter((phone_df.brand == 'Apple') | (phone_df.brand == 'Google')).show()

+---------+------+-----+----------+
|    model| brand|stock|unit_price|
+---------+------+-----+----------+
| iPhone 6| Apple|    6|     549.0|
|iPhone 6s| Apple|    5|     585.0|
| iPhone 7| Apple|   11|     739.0|
|    Pixel|Google|    8|     859.0|
| Pixel XL|Google|    2|     959.0|
+---------+------+-----+----------+



### Trier les lignes

Nous pouvons utiliser l'opération [`orderBy`](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.orderBy.html) pour trier les données :

In [11]:
phone_df.orderBy('unit_price').show()

+-------------+-------+-----+----------+
|        model|  brand|stock|unit_price|
+-------------+-------+-----+----------+
|    Galaxy A5|Samsung|    7|     297.0|
|    Galaxy S6|Samsung|    5|     414.0|
|    Galaxy S7|Samsung|   10|     539.0|
|     iPhone 6|  Apple|    6|     549.0|
|    iPhone 6s|  Apple|    5|     585.0|
|     iPhone 7|  Apple|   11|     739.0|
|Galaxy Note 7|Samsung|    0|     841.0|
|        Pixel| Google|    8|     859.0|
|     Pixel XL| Google|    2|     959.0|
+-------------+-------+-----+----------+



Dans la cellule suivante, nous utilisons une chaîne de méthodes DataFrame qui sont très similaires au langage de requête SQL utilisé pour certaines bases de données.
    Notez que nous n'utilisons que les noms des colonnes. Notez également l'utilisation de guillemets doubles et simples dans la méthode [`where`](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.where.html).

In [12]:
phone_df.select("model", "unit_price").where("brand='Apple'").orderBy('stock', ascending=False).show()

+---------+----------+
|    model|unit_price|
+---------+----------+
| iPhone 7|     739.0|
| iPhone 6|     549.0|
|iPhone 6s|     585.0|
+---------+----------+



Une autre façon de faire la même chose que la cellule ci-dessus est d'utiliser `phone_df[« brand »]` dans la clause where. C'est plus long à taper mais intuitivement plus clair et plus facile à lire. Il n'y a pas d'ambiguïté pour l'analyseur Spark avec cette notation.

In [ ]:
phone_df.select("model", "unit_price").where(phone_df["brand"]=="Apple").orderBy('stock', ascending=False).show()

## Exercice 3
Sélectionner tous les téléphones dont le prix unitaire est supérieur à 300 et dont il y a plus de deux en stock. Affichez les téléphones restants, classés par marque, puis par stock. Utilisez la syntaxe de spécification de colonne que vous préférez.

In [13]:
# TODO: Remplacer <FILL IN> avec le code approprié
<FILL IN>

SyntaxError: invalid syntax (<ipython-input-13-45a4860a2a25>, line 2)

 ## Agrégation des données
Une partie importante du traitement des données est la possibilité de combiner plusieurs enregistrements. Dans l'API DataFrame, il s'agit d'un processus en deux étapes :

D'abord, vous regroupez les données en utilisant la méthode `groupBy`. `groupBy` peut opérer sur une ou plusieurs colonnes. Elle n'effectue pas le regroupement mais crée une référence à un objet `GroupedData` :

In [14]:
grouped_df = phone_df.groupBy('brand')
print(type(grouped_df))

<class 'pyspark.sql.group.GroupedData'>


Une fois les données regroupées, nous pouvons leur appliquer l'une des fonctions d'agrégation standard. Elles sont répertoriées dans la documentation de l'API [GroupedData](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/grouping.html) de l'API. Il s'agit de : `min`, `max`, `mean`, `sum` et `count`.

Nous pouvons appliquer une agrégation à toutes les colonnes ou à un sous-ensemble de colonnes.

In [ ]:
# Minimum for all columns
min_df = grouped_df.min('unit_price')

min_df.toPandas()

Notez que `min(unit_price)` est le nom de la nouvelle colonne.

Si vous souhaitez renommer une colonne, utilisez [`withColumnRenamed`](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.withColumnRenamed.html). Cette méthode prend comme arguments l'ancien et le nouveau nom de la colonne.

Notez également qu'on a transformé le DataFrame Spark en DataFrame Pandas avec `toPandas`. Si on voulait rester 'pure Spark', on pourrait faire appel à `show()`, par exemple.

## Exercice 4

Calculez le maximum du prix unitaire par marque et renommez la colonne résultante en `max`.

Nous supposons que vous pouvez faire cela en une seule ligne. N'hésitez pas à adapter la cellule et à utiliser plus de lignes si vous le souhaitez.


In [17]:
# TODO: Remplacer <FILL IN> avec le code approprié
max_df = <FILL IN>
max_df.toPandas()

+-------+---------------+
|  brand|max(unit_price)|
+-------+---------------+
| Google|          959.0|
|  Apple|          739.0|
|Samsung|          841.0|
+-------+---------------+



Dans certains cas, on souhaite effectuer des aggrégations différentes sur plusieurs colonnes. Dans ce cas, nous pouvons combiner les différentes agrégations par colonne en utilisant la méthode [`agg`](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.GroupedData.agg) sur une instance de GroupedData :

In [18]:
# Fait la somme de la colonne stock et la moyenne de la colonne unit_price
sum_df = grouped_df.agg({'stock': 'sum', 'unit_price': 'mean'})

sum_df.show()

+-------+----------+-----------------+
|  brand|sum(stock)|  avg(unit_price)|
+-------+----------+-----------------+
| Google|        10|            909.0|
|  Apple|        22|624.3333333333334|
|Samsung|        22|           522.75|
+-------+----------+-----------------+



## SQL
L'API SQL permet d'interroger les dataframes avec une syntaxe compatible SQL (utilisée dans la plupart des bases de données). Vous pouvez accéder à l'API SQL à partir de la session Spark en utilisant `spark.sql`. Voici comparaison entre une requête effectuée en utilisant l'API DataFrame de Spark et une requête avec l'API SQL :

In [19]:
# DataFrame version
res_df = phone_df.filter(phone_df['stock'] > 7).select('model')
res_df.show()

+---------+
|    model|
+---------+
| iPhone 7|
|    Pixel|
|Galaxy S7|
+---------+



La version SQL de la requête exige que nous « enregistrions » le DataFrame en tant que table SQL :

In [20]:
# SQL version

# Register the phone_df DataFrame within SQL as a table with name 'phones'
phone_df.createOrReplaceTempView('phones')

# Perform the SQL query on the 'phones' table
res_df = spark.sql('SELECT model FROM phones WHERE stock > 7')
res_df.show()

+---------+
|    model|
+---------+
| iPhone 7|
|    Pixel|
|Galaxy S7|
+---------+



## Jointure avec d'autres ensembles de données
Il arrive souvent que vous souhaitiez combiner plusieurs ensembles de données en se basant sur une même colonne (clé commune). Dans cet exemple avec l'API Spark DataFrame, nous créons une table supplémentaire contenant des informations sur le fabricant du téléphone :

In [21]:
companies = [
    ('Google', 'USA', 1998, 'Sundar Pichai'),
    ('Samsung', 'South Korea', 1938 ,'Oh-Hyun Kwon' ),
    ('Apple', 'USA', 1976 ,'Tim Cook')
]

columns = ['company_name', 'hq_country', 'founding_year', 'ceo']

company_df = spark.createDataFrame(companies, columns)
company_df.show()

+------------+-----------+-------------+-------------+
|company_name| hq_country|founding_year|          ceo|
+------------+-----------+-------------+-------------+
|      Google|        USA|         1998|Sundar Pichai|
|     Samsung|South Korea|         1938| Oh-Hyun Kwon|
|       Apple|        USA|         1976|     Tim Cook|
+------------+-----------+-------------+-------------+



Pour joindre deux DataFrames, nous utilisons la méthode `join` sur l'un des DataFrames. Cette méthode prend deux arguments :  

1.   l'autre DataFrame, et
2.   une relation de jointure.

Ici, nous joignons les deux DataFrames sur les colonnes brand/company_name :

In [22]:
joined_df = phone_df.join(company_df, phone_df['brand'] == company_df['company_name'])
joined_df.show()

+-------------+-------+-----+----------+------------+-----------+-------------+-------------+
|        model|  brand|stock|unit_price|company_name| hq_country|founding_year|          ceo|
+-------------+-------+-----+----------+------------+-----------+-------------+-------------+
|     iPhone 6|  Apple|    6|     549.0|       Apple|        USA|         1976|     Tim Cook|
|    iPhone 6s|  Apple|    5|     585.0|       Apple|        USA|         1976|     Tim Cook|
|     iPhone 7|  Apple|   11|     739.0|       Apple|        USA|         1976|     Tim Cook|
|        Pixel| Google|    8|     859.0|      Google|        USA|         1998|Sundar Pichai|
|     Pixel XL| Google|    2|     959.0|      Google|        USA|         1998|Sundar Pichai|
|    Galaxy S7|Samsung|   10|     539.0|     Samsung|South Korea|         1938| Oh-Hyun Kwon|
|    Galaxy S6|Samsung|    5|     414.0|     Samsung|South Korea|         1938| Oh-Hyun Kwon|
|    Galaxy A5|Samsung|    7|     297.0|     Samsung|South K

Ici, nous avons la même jointure en utilisant SQL. Notez qu'on doit également déclarer le dataframe `company_df` en tant que table (avec le nom `company`).

In [24]:
# Register the phone_df DataFrame within SQL as a table with name 'phones'
company_df.createOrReplaceTempView('company')

# Perform the SQL query on the 'phones' table
join_df = spark.sql('SELECT * FROM phones JOIN company ON phones.brand = company.company_name')
join_df.show()

+-------------+-------+-----+----------+------------+-----------+-------------+-------------+
|        model|  brand|stock|unit_price|company_name| hq_country|founding_year|          ceo|
+-------------+-------+-----+----------+------------+-----------+-------------+-------------+
|     iPhone 6|  Apple|    6|     549.0|       Apple|        USA|         1976|     Tim Cook|
|    iPhone 6s|  Apple|    5|     585.0|       Apple|        USA|         1976|     Tim Cook|
|     iPhone 7|  Apple|   11|     739.0|       Apple|        USA|         1976|     Tim Cook|
|        Pixel| Google|    8|     859.0|      Google|        USA|         1998|Sundar Pichai|
|     Pixel XL| Google|    2|     959.0|      Google|        USA|         1998|Sundar Pichai|
|    Galaxy S7|Samsung|   10|     539.0|     Samsung|South Korea|         1938| Oh-Hyun Kwon|
|    Galaxy S6|Samsung|    5|     414.0|     Samsung|South Korea|         1938| Oh-Hyun Kwon|
|    Galaxy A5|Samsung|    7|     297.0|     Samsung|South K

Here is an example of a more complicated query that combines multiple steps:

In [ ]:
# All the models from USA companies with more than 7 items in stock
result = phone_df \
    .join(company_df, phone_df['brand'] == company_df['company_name']) \
    .filter(company_df['hq_country'] == 'USA') \
    .filter(phone_df['stock'] > 7) \
    .select('model')

result.show()

# Lecture de fichiers/sources structurés
L'un des avantages des DataFrames est la possibilité de lire des données déjà structurées et d'importer automatiquement la structure dans Spark. Spark contient des lecteurs pour un certain nombre de formats tels que csv, json, parquet, orc, text et jdbc. Il existe également des lecteurs/connecteurs tiers pour des bases de données telles que MongoDB et Cassandra.

Dans cet exemple, nous allons lire des tweets au format json. Comme vous pouvez le voir, le schéma du fichier JSON est déduit automatiquement.

In [25]:
!wget https://github.com/lsteffenel/pyspark-binder/raw/master/tweets.json

--2025-03-02 18:37:00--  https://github.com/lsteffenel/pyspark-binder/raw/master/tweets.json
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/lsteffenel/pyspark-binder/master/tweets.json [following]
--2025-03-02 18:37:01--  https://raw.githubusercontent.com/lsteffenel/pyspark-binder/master/tweets.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1629848 (1.6M) [text/plain]
Saving to: ‘tweets.json’

tweets.json         100%[===================>]   1.55M  --.-KB/s    in 0.07s   

2025-03-02 18:37:01 (21.9 MB/s) - ‘tweets.json’ saved [1629848/1629848]



In [26]:
tweet_df = spark.read.format("json").load('tweets.json')
tweet_df.printSchema()

root
 |-- country: string (nullable = true)
 |-- id: string (nullable = true)
 |-- place: string (nullable = true)
 |-- text: string (nullable = true)
 |-- user: string (nullable = true)



Cette structure est comprimée dans un tableau, nous pouvons voir à quoi ressemble le premier tweet dans un DataFrame.

In [27]:
tweet_df.show(1)

+-------+------------------+------+--------------------+---------------+
|country|                id| place|                text|           user|
+-------+------------------+------+--------------------+---------------+
|  India|572692378957430785|Orissa|@always_nidhi @Yo...|Srkian_nishu :)|
+-------+------------------+------+--------------------+---------------+
only showing top 1 row



In [29]:
tweet_df.toPandas().head(1)

,country,id,place,text,user
0,India,572692378957430785,Orissa,@always_nidhi @YouTube no i dnt understand bt ...,Srkian_nishu :)


## Exercice 5
Sélectionnez le nom et le nom d'écran de l'utilisateur, le champ texte et le champ lang.

**Astuce** : les champs imbriqués peuvent être sélectionnés en utilisant la notation par points, c'est-à-dire `df.select('<parent>.<child>')`.

In [ ]:
name_df = tweet_df.<FILL IN>
name_df.toPandas().head(15)

## Assignment 7
Count the number of tweets per user, and display the top 10 most-tweeting users.

In [ ]:
<FILL IN>

## Word count in DataFrames

It is also possible to use DataFrames for less-structured data such as text. Here we show how you could do word count with DataFrames.

The following chained query contains a number of methods you haven't seen before, and we'll go through it line by line.

In [ ]:
!wget https://github.com/lsteffenel/pyspark-binder/raw/master/shakespeare.txt

In [ ]:
from pyspark.sql.functions import explode, split

spark \
    .read.text('shakespeare.txt') \
    .select(explode(split("value", "\W+")).alias("word")) \
    .groupBy("word") \
    .count() \
    .orderBy("count", ascending=0).show()

To see what happens here, we break it down into steps. First we read in the data file and inspect the DataFrame. It contains one column, called `value` by default.

In [ ]:
swan_df = spark.read.text('shakespeare.txt')
swan_df.show()

The column name `value` explains why it is mentioned inside the `split` function. Let's call the `select` method but omit `explode` and see what happens. Notice, that with `alias` we rename the column.

In [ ]:
split_df = swan_df.select(split("value", "\W+").alias("word"))
split_df.show()

Looking at the schema, we can see that `word` is actually an array of strings:

In [ ]:
split_df.printSchema()

Instead, we would like to have a row for each word, which is where [`explode`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.explode) comes in. It has a similar meaning as `flatMap` in Spark RDDs. It gets rid of lists:

In [ ]:
swan_df.select(explode(split("value", "\W+")).alias("word")).show()

### User-defined functions

In the previous example we used the built-in split function. It is also possible to define and use a custom user-defined function, or UDF. We'll show an example for the phone stock DataFrame first:

In [ ]:
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf

exp_udf = udf(lambda price: "Expensive" if price >= 500 else "Inexpensive", StringType())

phone_df.withColumn("cost", exp_udf(phone_df['unit_price'])).show()

In this manner, we can apply specialized function, like tokenizers, on DataFrames. However, we first must register them as UDFs and cannot simply define them inline with lambda functions like we can with RDDs.

Below we define a very simple tokenizer, just as an example. It uses Python's string `split`, and also lowers the case of the text.

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType

def my_tokenize(s):
    s = s.lower()
    words = s.split()
    return words

returnType = ArrayType(StringType())

tokenize_udf = udf(my_tokenize, returnType)

## Assignment 8
Use the `my_tokenize` function from the last cell to count words on the Shakespeare DataFrame `swan_df` instead of usng the `split` function. Display the top 10 most occurring words.

In [ ]:
<FILL IN>